In [17]:
import numpy as np
import gdal
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl


def ADAM(t11, t22, t33, t12_i, t12_r):
    X1=t33/(t11-t33)
    X2=t33/t22
    gamma = 0
    if (X1>X2):
        X=X1
    else:
        X=X2

    A=t11*t22-t22* t33-np.square(t12_i)-np.square(t12_r)
    B=t22+t11-t33
    C=4*A + np.square(B)
    ROOT = (-B+np.sqrt(np.abs(C)))/2
    N_root = t33/ROOT
    if C>0 and X1>0 and N_root>0:
        if (N_root>X):
            gamma=N_root
        else:
            gamma=X
                    
        a = gamma+1
        b = 1
        c = gamma
        plus = a+b+c
    else:                  
        a = 1
        b = 0
        c = 1
        plus = a+b+c
    x11=t11-a*t33/c
    x22=t22-b*t33/c
    r13= a/c
    r23= b/c
    return (r13, r23, x11, x22, gamma)


def FDD(r13, r23 , x11, t12_r, t12_i, x22, t33):
    Pv = (r13+r23+1) * t33
    if (x11 > x22):
        alpha = 0
        Ps = x11 + (t12_i**2 + t12_r**2)/x11
        Pd = x22 - (t12_i**2 + t12_r**2)/x11
    else:
        beta = 0
        Ps = x11 - (t12_i**2 + t12_r**2)/x22
        Pd = x22 + (t12_i**2 + t12_r**2)/x22
    return(Ps, Pd, Pv)


###7CD
def GY7CD(t11, t12_r, t12_i, t13_r, t13_i, t22, t23_r, t23_i, t33):
    x11 = t11 - np.abs(t13_r) - np.abs(t13_i)
    x22 = t22 - np.abs(t23_i) - np.abs(t23_r)
    x33 = t33 - np.abs(t23_i) - np.abs(t13_r) - np.abs(t13_i)- np.abs(t23_r)
    Ph = 2* np.abs(t23_i)
    Pmd = 2* np.abs(t23_r)
    Pod = 2* np.abs(t13_r)
    Pcd = 2* np.abs(t13_i)
    return(x11, x22, x33, Ph, Pmd, Pod, Pcd)

def ADAM_process(input_processing_dir,OrderInlist = 0):
    processing_dir = input_processing_dir
    mylist = os.listdir(processing_dir)
    #print(len(mylist))
    os.chdir(processing_dir)


    ds = gdal.Open(mylist[OrderInlist], gdal.GA_ReadOnly)
    t11 = ds.GetRasterBand(1).ReadAsArray()
    t12_i = ds.GetRasterBand(2).ReadAsArray()
    t12_r = ds.GetRasterBand(3).ReadAsArray()
    t13_i = ds.GetRasterBand(4).ReadAsArray()
    t13_r = ds.GetRasterBand(5).ReadAsArray()
    t22 = ds.GetRasterBand(6).ReadAsArray()
    t23_i = ds.GetRasterBand(7).ReadAsArray()
    t23_r = ds.GetRasterBand(8).ReadAsArray()
    t33 = ds.GetRasterBand(9).ReadAsArray()
    #IncidenceAngle = ds.GetRasterBand(10).ReadAsArray()

    rows = len(t11)
    columns = len(t11[0])
    
    Ps = np.tile(0.0,[rows,columns])
    Pd = np.tile(0.0,[rows,columns])
    Pv = np.tile(0.0,[rows,columns])
    gamma = np.tile(0.0,[rows,columns])
    
    for i in np.arange(rows):
        for j in np.arange(columns):
            r13, r23, x11, x22, gamma[i,j] = ADAM(t11[i,j], t22[i,j], t33[i,j], t12_i[i,j], t12_r[i,j])
            Ps[i,j], Pd[i,j], Pv[i,j] = FDD(r13, r23 , x11, t12_r[i,j], t12_i[i,j], x22, t33[i,j])
    return (Ps, Pd, Pv, gamma)



def GY7CD_process(input_processing_dir,OrderInlist = 0):
    processing_dir = input_processing_dir
    mylist = os.listdir(processing_dir)
    #print(len(mylist))
    os.chdir(processing_dir)

    ds = gdal.Open(mylist[OrderInlist], gdal.GA_ReadOnly)
    t11 = ds.GetRasterBand(1).ReadAsArray()
    t12_i = ds.GetRasterBand(2).ReadAsArray()
    t12_r = ds.GetRasterBand(3).ReadAsArray()
    t13_i = ds.GetRasterBand(4).ReadAsArray()
    t13_r = ds.GetRasterBand(5).ReadAsArray()
    t22 = ds.GetRasterBand(6).ReadAsArray()
    t23_i = ds.GetRasterBand(7).ReadAsArray()
    t23_r = ds.GetRasterBand(8).ReadAsArray()
    t33 = ds.GetRasterBand(9).ReadAsArray()
    #IncidenceAngle = ds.GetRasterBand(10).ReadAsArray()

    rows = len(t11)
    columns = len(t11[0])
    
    Ps = np.tile(0.0,[rows,columns])
    Pd = np.tile(0.0,[rows,columns])
    Pv = np.tile(0.0,[rows,columns])
    Ph = np.tile(0.0,[rows,columns])
    Pmd = np.tile(0.0,[rows,columns])
    Pod = np.tile(0.0,[rows,columns])
    Pcd = np.tile(0.0,[rows,columns])
    
    r13 = 15/8
    r23 = 7/8
    
    for i in np.arange(rows):
        for j in np.arange(columns):
            x11, x22, x33, Ph[i,j], Pmd[i,j], Pod[i,j], Pcd[i,j] = GY7CD(t11[i,j], t12_r[i,j], t12_i[i,j], t13_r[i,j], t13_i[i,j], t22[i,j], t23_r[i,j], t23_i[i,j], t33[i,j])
            Ps[i,j], Pd[i,j], Pv[i,j] = FDD(r13, r23 , x11, t12_r[i,j], t12_i[i,j], x22, x33)
 
    return (Ps, Pd, Pv, Ph, Pmd, Pod, Pcd)

def FDDalgorithm(input_processing_dir,OrderInlist = 0):

    processing_dir = input_processing_dir
    mylist = os.listdir(processing_dir)
    #print(len(mylist))
    os.chdir(processing_dir)

    ds = gdal.Open(mylist[OrderInlist], gdal.GA_ReadOnly)
    t11 = ds.GetRasterBand(1).ReadAsArray()
    t12_i = ds.GetRasterBand(2).ReadAsArray()
    t12_r = ds.GetRasterBand(3).ReadAsArray()
    t13_i = ds.GetRasterBand(4).ReadAsArray()
    t13_r = ds.GetRasterBand(5).ReadAsArray()
    t22 = ds.GetRasterBand(6).ReadAsArray()
    t23_i = ds.GetRasterBand(7).ReadAsArray()
    t23_r = ds.GetRasterBand(8).ReadAsArray()
    t33 = ds.GetRasterBand(9).ReadAsArray()
    #IncidenceAngle = ds.GetRasterBand(10).ReadAsArray()

    rows = len(t11)
    columns = len(t11[0])

    x11 = t11-2.0*t33;
    x22 = t22-1.0*t33;
    x12 = np.where(((x11>0)&(x22>0)),x11*x22-np.square(t12_r)-np.square(t12_i),0)

    Ps = np.tile(0.0,[rows,columns])
    Pd = np.tile(0.0,[rows,columns])
    Pv = np.tile(0.0,[rows,columns])

    for i in np.arange(rows):
        for j in np.arange(columns):
            cat_a = 2.0;
            cat_b = 1.0;            
            #print(3)
            cat_x11 = t11[i,j]-cat_a*t33[i,j]
            cat_x22 = t22[i,j]-cat_b*t33[i,j]
            Ps[i,j], Pd[i,j], Pv[i,j] = FDD(cat_a, cat_b, cat_x11, t12_r[i,j], t12_i[i,j], cat_x22, t33[i,j])
    return (Ps, Pd, Pv)


In [18]:
path = '/media/lengmeqin/wzz/Wuhan/ALOS10/tif'
OL = 6
Psf, Pdf, Pvf = FDDalgorithm(path,OrderInlist = OL)
Ps, Pd, Pv, gamma = ADAM_process(path,OrderInlist = OL)
Psy, Pdy, Pvy, Ph, Pmd, Pod, Pcd = GY7CD_process(path,OrderInlist = OL)